In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import re

In [2]:
#Make a copy of the dataset so we have all the original information as we manipulate columns and values

df = pd.read_csv('cc_capstone/profiles.csv')
df_copy = df.copy()

FileNotFoundError: [Errno 2] File cc_capstone/profiles.csv does not exist: 'cc_capstone/profiles.csv'

In [ ]:
df_copy.columns

## Create a new dataframe with non-essay columns

In [ ]:
df_chars = df_copy[['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'ethnicity', 'height', 'income', 'job',
                   'last_online', 'location', 'offspring', 'orientation', 'pets', 'religion', 'sex', 'sign', 'smokes', 
                   'speaks', 'status']]
df_chars.head()

## Summarize and get a feel for the data
- Find how many samples exist in the dataset
- Understand data type of each feature and how to best manipulate it to suit our interest
(df.info is more comprehensive than df.dtypes)     

In [ ]:
df_features = df_copy.columns
print(df_features)
df_features.size
df.info()

In [ ]:
df_copy.shape
#Almost 60,000 observations!

## Correlation Table between numeric data -- Encode all other features

In [ ]:
df_chars.corr()

In [ ]:
df.age.describe()

In [ ]:
df.height.describe()

In [ ]:
df_copy.income.describe()

In [ ]:
df_copy.income.value_counts()
unreported = (59946 - 48442)/(59946)*100

In [ ]:
print("With only {x:.2f}% of data NOT EQUAL to -1 & avaiable, one should DROP the feature.".format(x = unreported))

## Create a drop list and append as you find features that should be dropped

In [ ]:
init_drop_cols = ['income']

In [ ]:
nonnum_df = df_chars.drop(['age', 'height', 'income'], axis = 1)
nonnum_df.head(3)

## Treat Status as Target Feature

In [ ]:
df_chars.status.unique()

In [ ]:
df_chars.status.value_counts()

### Action Items for 'Status' column
- <b/>Create</b> Single: 'single' + 'available'
- <b/>Create</b> Taken: 'seeing someone'
- <b/>Remove</b>: 'married' + 'unknown'

# Definitions and thoughts -- in an accompanied Google Sheets
- body_type:
- diet:
- drinks:
- drugs:
- education:

- ethnicity:
- height:
- income:
- job:
- offspring:
- orientation:
- pets:
- religion:
- sex:
- sign:
- smokes:
- speaks:
- status:


# Find the amount of missing data by feature

In [ ]:
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/ df.isnull().count()).sort_values(ascending = False)
    missing_data = pd.concat([total, percent], axis = 1, keys = ['Total','Percent Missing'])
    return missing_data

In [ ]:
draw_missing_data_table(df_copy)

### Drop Features with insufficient data; later build or average values for NA
- offspring
- diet
- **pets**

In [ ]:
init_drop_cols.append(['offspring', 'diet', 'religion'])

In [ ]:
essay_cols = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9']

In [ ]:
essay_dat = {'Essay Summary': ["My self summary", 
             "What I’m doing with my life", 
             "I’m really good at",
             "The first thing people usually notice about me",
             "Favorite books, movies, show, music, and food",
             "The six things I could never do without",
             "I spend a lot of time thinking about",
             "On a typical Friday night I am",
             "The most private thing I am willing to admit",
             "You should message me if…"]}

In [ ]:
essay_missing = draw_missing_data_table(df_copy).loc[essay_cols, :]
essay_missing

In [ ]:
essay_summary = pd.DataFrame(data = essay_dat, index = essay_cols)
essay_summary = essay_summary.join(essay_missing)
essay_summary.sort_values(by = 'Percent Missing', ascending = False)

## Create dataframe without Essay Columns

In [ ]:
non_essay_feats = df_copy[df_copy.columns.difference(essay_cols)]
non_essay_feats.head(3)

In [ ]:
init_drop_cols

In [ ]:
non_essay_feats = non_essay_feats[non_essay_feats.columns.difference(['income', 'offspring', 'diet'])]
non_essay_feats

## Create a table to explore features you haven't already

In [ ]:
feats_to_explore = non_essay_feats[non_essay_feats.columns.difference(['age', 
                                                                       'height', 
                                                                       'last_online', 
                                                                       'location', 
                                                                       'orientation', 
                                                                       'sex',])]
feats_to_explore.head(4)

In [ ]:
numfeats2encode_df = nonnum_df[['body_type', 'drinks', 'drugs', 'education', 'smokes']]
catfeats2encode_df = nonnum_df[['diet', 'ethnicity', 'job', 'location', 'offspring',
                                'orientation', 'pets', 'religion', 'sign', 'speaks' ]] 
#include cat feature whether currently in education

### Create user_id column for each row; helps to group data by

In [ ]:
df_copy['user_id'] = range(len(df))
df_copy.tail()

# Split Location data
-> City and State Features
<br />
then ->: California and Other States

In [ ]:
location_table = df_copy.location.str.split(pat = ',', expand = True)
del location_table[2]
location_table.rename(columns = {0: 'City', 1: 'State'}, inplace = True)
print(location_table)

## Create a Table of the Total Survey Respondants by Unique Cities

In [ ]:
type(location_table['City'].unique())

In [ ]:
cities = location_table.City
(unq_city, city_count) = np.unique(cities, return_counts = True)
unique_freq = np.asarray((unq_city, city_count)).T
unique_freq.sort(axis = 0)
unique_freq

In [ ]:
y = pd.DataFrame(unique_freq)
y.sort_values(by = 1, ascending = False).reset_index()
y.columns = ['City', 'Survey Count']
z = y.sort_values(by = 'Survey Count', ascending = False).reset_index(drop = True) #drop attribute removes the index
z

In [ ]:
z["Cummulative Sum"] = z['Survey Count'].cumsum()
z

#### Create a Percent of Total Survey column

In [ ]:
z["Percent of Total Survey Respondants"] = z['Cummulative Sum']/ z['Cummulative Sum'][197]*100
z

### By reducing the variety of locations by 134 one only removes 0.5% of the Survey Respondants !

In [ ]:
neglig_data = z[z.index > 63]
neglig_data["Survey Count"].sum()
neglig_data

### Find the Count of the Unique states

In [ ]:
states = location_table.State
(unq_state, state_count) = np.unique(states, return_counts = True)
unique_states = np.asarray((unq_state, state_count)).T
unique_states

In [ ]:
z = pd.DataFrame(unique_states)
z.sort_values(by = 1, ascending = False).reset_index()
z.columns = ['State', 'Survey Count']
z.sort_values(by = 'Survey Count', ascending = False).reset_index(drop = True) #drop attribute removes the index

In [ ]:
''' Select Rows in the dataFrame that only contain California as a location'''
non_essay_feats = non_essay_feats[non_essay_feats.location.str.contains('california')]
non_essay_feats.location #Notice the length matches the Survey Count found above

## Notice some Surveys are from outside the COUNTRY - not just the state:
- United Kingdom (2)
- Spain (2)
- Vietnam (1)
- Switzerland (1)
- Netherlands (1)
- Brittish Columbia (1)
- Ireland (1)
- Germany (1)<br>**TOTAL: 10**

## Timeframe of the Data:
 + 2011 - 2012
 + End of June to Beginning of July (~Essentially JULY2011 - July2012)

In [ ]:
df_copy['last_online'] = df_copy['last_online'].astype('string') # Change dtype to a string to convert into Datetime
df_copy.last_online

### Create a datetime object to parse the last_online values


In [ ]:
df_copy['last_online'] = pd.to_datetime(df['last_online'], format = "%Y-%m-%d-%H-%M")
df_copy.last_online #access datetime features w/ df.feature.dt.____(month, date, time, etc.)

## Create a Table of Dates and Times that users Logged on

In [ ]:
dt_table = pd.DataFrame()
dt_table['Date'] = df_copy['last_online'].dt.date
dt_table['Time'] = df_copy['last_online'].dt.time
sorted = dt_table.sort_values(by = 'Date')
sorted

In [ ]:
dt_table['Time'] # under 24 hr time

### Find the most Frequent Hours users logged on

In [ ]:
hours = [dt_table.Time[x].hour for x in range(len(dt_table.Time))]

## Create a function to count the Hour occurences from the times list

In [ ]:
def countFreq(list):
    freq_dic = {}
    for hour in list:
        freq_dic[hour] = list.count(hour)
    return(freq_dic)

In [ ]:
c = countFreq(hours)
c

In [ ]:
from sortedcontainers import SortedDict
sort_hours = SortedDict(c)
sort_hours

In [ ]:
x = list(sort_hours.keys())
y = list(sort_hours.values())
plt.bar(x = x, height = y, edgecolor = "black", tick_label = x)
plt.title("Most Frequent log-in Hours for Users")
plt.tight_layout()

## Analyze Information:
- Dates
    - Most Frequent months
    - Most Frequent Weekday
- Times
    - Which 1/3 of the Day is most popular: Morning, Afternoon, Evening
    - Popular time by the hour

In [ ]:
dates = dt_table.Date
(unq_date, date_count) = np.unique(dates, return_counts = True)
unique_dates = np.asarray((unq_date, date_count)).T
unique_dates[:25]

In [ ]:
dt = pd.DataFrame(unique_dates, columns = ['Dates', 'Frequency'])

### Datetime Table sorted from Past to most Recent

In [ ]:
dt_order = dt.sort_values(by = 'Dates', ascending = True) #Dates from past to most recent
dt_order

In [ ]:
plt.plot_date(x = dt_order['Dates'], y = dt_order['Frequency'])
plt.xlabel('Dates')
plt.ylabel('Last-Online Frequency')
plt.title("Last-Online Frequency by Date")

## Datetime Table sorted by most Frequent dates users logged-on: 
- *Find the table values to be consistent with the graph shown above!*

In [ ]:
dt.sort_values(by = 'Frequency', ascending = False).reset_index(drop = True) #Freq from HIGH to Low

### Explore the Growth stage of the dates

In [ ]:
low_freq_dt = dt_order[0:355]
low_freq_dt
plt.plot_date(x = low_freq_dt['Dates'], y = low_freq_dt['Frequency'])
plt.xlabel('Dates')
plt.ylabel('Last-Online Frequency')
plt.title("Last-Online Frequency by Date")

# EssayQuestions include:
- essay0 - My self summary
- essay1 - What I’m doing with my life
- essay2 - I’m really good at
- essay3 - The first thing people usually notice about me
- essay4 - Favorite books, movies, show, music, and food
- essay5 - The six things I could never do without
- essay6 - I spend a lot of time thinking about
- essay7 - On a typical Friday night I am
- essay8 - The most private thing I am willing to admit
- essay9 - You should message me if…


## Correlation Matrix between Numerical Features

In [ ]:
corr = df_copy.corr()
ax = sns.heatmap(corr, vmin = -1, vmax= 1, center = 0, cmap = sns.diverging_palette(20,220, n=200), square = True)
ax.set_xticklabels(
    ax.get_xticklabels(), rotation= 45, horizontalalignment = 'right'
)

## EDA Bar charts for certain features

In [ ]:
plt.title('Ages of Survey Respondants')
plt.hist(df_copy.age, bins = 30)
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.xlim(16,70)

# plt.boxplot(df_copy.age)
plt.show()

In [ ]:
plt.hist(df_copy.sex, bins = 2, align = 'mid')
plt.xlabel('sex')
plt.ylabel('freq')
plt.title('Distribution of Male and Female participants')
plt.show()

In [ ]:
plt.hist(df_copy.orientation, bins = 3)
plt.xlabel('orientation')
plt.ylabel('Frequency')
plt.title('Distribution of Sexual Orientation in Data')
plt.tight_layout()
plt.show()

In [ ]:
plt.hist(df_copy.location, bins = 20, orientation = 'vertical')
plt.xlabel('locations')
plt.ylabel('freq')
plt.title('Frequency of Responses based on Location')
# plt.xticks(rotation = 'vertical')
plt.tight_layout()
plt.show()

# Features to still synthesize:
- Education
- Job
- Religion
- Pets
- Sign/ Zodiac
- Status

In [ ]:
df_copy['education'].nunique()

In [ ]:
unique_edu = df_copy['education'].unique()
unique_edu

## Create a dictionary that maps the 32 different education features to 4:
- Graduated High School
- Graduated College/ University
- Graduated Masters or PhD program
- Other

In [ ]:
loc_data = df_copy.groupby(['location', 'sex'])['sex'].count().sort_values(ascending = False)
print(loc_data.head(60))
loc_data.to_frame()
print(loc_data['sex'])
location_data = df_copy.groupby(['location'])['sex'].count().sort_values(ascending = False)

In [ ]:
df_copy['ethnicity'].unique()
df_copy['ethnicity'].dropna()

In [ ]:
df_copy['ethnicity'].nunique()

In [ ]:
df_copy['status'].unique()

In [ ]:
plt.rcParams['figure.figsize'] = [20,20]
plt.rcParams['axes.labelsize'] = 50
plt.rcParams['xtick.direction'] = 'inout'
plt.rcParams['legend.fontsize'] = 25
for i in init_drop_cols:
    df_copy[init_drop_cols][i].hist()
plt.tight_layout()
plt.legend(init_drop_cols)
# df_copy['offspring'].hist()

In [ ]:
df_copy.diet.value_counts()

In [ ]:
df_copy.drugs.value_counts()

In [ ]:
df_copy.drinks.value_counts()

In [ ]:
df_copy.education.value_counts()
# edu_table = pd.DataFrame(['In School', 
#                           'Graduated', 
#                           ''])

In [ ]:
df_copy.ethnicity.value_counts()

In [ ]:
df_copy.income.value_counts()

In [ ]:
df_copy.job.value_counts()

In [ ]:
df_copy.last_online.value_counts().head(50)

In [ ]:
df_copy.location.value_counts().head(50)
# df_copy.location.value_counts().head(50).sum()

In [ ]:
df_copy.pets.value_counts(dropna = False)

In [ ]:
zodiac_table = pd.DataFrame(columns = ['Leo',
                                       'Libra', 
                                       'Cancer', 
                                       'Virgo', 
                                       'Scorpio', 
                                       'Gemini', 
                                       'Taurus', 
                                       'Aries', 
                                       'Piscs', 
                                       'Aquarius', 
                                       'Sagittarius', 
                                       'Capricorn', 
                                       'Matters'])
#Matters is a boolean value of 1 or 0 -- Y or N

In [ ]:
df_copy.sign.value_counts(dropna = False) #44 value_counts /48

In [ ]:
zodiac_dict = {'leo': 0, 'libra': 0, 'cancer': 0, 'virgo': 0, 'scorpio': 0, 'gemini': 0, 'taurus': 0, 'aries': 0, 'pisces': 0, 'aquarius': 0, 'sagittarius': 0, 'capricorn': 0}

### Formulate a Question
df.sign.value_counts()

** Still have to drop na values for features + drop some features themselves

### Augment Your data - by mapping categorical data -> numerical

done to drinks, smokes, and drugs columns

 drink_mapping = {"not at all": 0, "rarely": 1, "socially": 2, "often": 3, "very often": 4, "desperately": 5}

all_data["drinks_code"] = all_data.drinks.map(drink_mapping)




In [ ]:
location_names = df_copy.groupby(['location']).groups
print(location_names.items())


In [ ]:
locations = df_copy.groupby(['location', 'sex'], as_index = False).count()
print(locations)

### Calculate how many locations are in california and otherwise

In [ ]:
location_frame = df_copy.location.to_frame()
print(type(df_copy))
# for i in range(len(location_frame)):
#     cali_count = 0
#     other = 0
#     if location_frame.str.contains(', california', na = False):
#         cali_count += 1
#     else:
#         other += 1
# print(cali_count)

In [ ]:
print(location_data_total)

In [ ]:
location_labels = df_copy.groupby(['location'])['sex']
print(location_labels)

In [ ]:
plt.pie(location_data, autopct = '%d%%')
plt.legend(location_data)
plt.show()